This is a simple ipython notebook that does a few gradient checks for common operators. There is nothing much useful here - unless you would like to do some debugging. Due to the fact that the gradient checker is written in Python, there are a lot of communications going on so the whole check runs relatively slowly. Run at your own risk, and in the meantime, go grab a coffee. If everything is right, this should output all Trues. If some gradient is wrong, the script will print out the estimated and the computed gradients for your inspection.

In [1]:
%matplotlib inline
import numpy as np
from pycaffe2 import core, device_checker, gradient_checker, workspace
from caffe2.proto import caffe2_pb2, caffe2_legacy_pb2
from matplotlib import pyplot

import sys

device_option = caffe2_pb2.DeviceOption()
device_option.device_type = caffe2_pb2.CUDA

cpu_device_option = caffe2_pb2.DeviceOption()
device_checker = device_checker.DeviceChecker(0.01, [device_option, cpu_device_option])
gpu_g_checker = gradient_checker.GradientChecker(0.005, 0.05, device_option, "gpu_checker_ws")
cpu_g_checker = gradient_checker.GradientChecker(0.01, 0.05, cpu_device_option, "cpu_checker_ws")

loaded nvd3 IPython extension
run nvd3.IPython_wrapper.initialize_javascript() to set up the notebook
help(nvd3.IPython_wrapper.initialize_javascript) for options


In [2]:
# This is for convolution layers with NHWC order.
test_configs = [
(1, 1, 1, 7, "NHWC"),
(1, 1, 2, 7, "NHWC"),
(1, 3, 1, 7, "NHWC"),
(1, 3, 2, 7, "NHWC"),
(1, 5, 1, 14, "NHWC"),
(1, 5, 2, 14, "NHWC"),
(2, 7, 1, 24, "NHWC"),
(2, 7, 2, 24, "NHWC"),
(1, 1, 1, 7, "NCHW"),
(1, 1, 2, 7, "NCHW"),
(1, 3, 1, 7, "NCHW"),
(1, 3, 2, 7, "NCHW"),
(1, 5, 1, 14, "NCHW"),
(1, 5, 2, 14, "NCHW"),
(2, 7, 1, 24, "NCHW"),
(2, 7, 2, 24, "NCHW"),
]

for stride, kernel, legacy_pad, size, order in test_configs:
    op = core.CreateOperator("Conv")(
        ["X", "w", "b"], ["Y"], stride=stride, kernel=kernel,
        legacy_pad=legacy_pad, order=order, device_option=device_option)
    if order == "NHWC":
        X = np.random.rand(2, size, size, 3).astype(np.float32) - 0.5
        w = np.random.rand(4, kernel, kernel, 3).astype(np.float32) - 0.5
    else:
        X = np.random.rand(2, 3, size, size).astype(np.float32) - 0.5
        w = np.random.rand(4, 3, kernel, kernel).astype(np.float32) - 0.5        
    b = np.random.rand(4).astype(np.float32) - 0.5
    res = device_checker.CheckSimple(op, [X, w, b], [0])
    print 'Device test for', stride, kernel, legacy_pad, size, order, 'is: ', res
    res, grad, grad_estimated = cpu_g_checker.CheckSimple(op, [X, w, b], 0, [0])
    print 'Test for cpu', stride, kernel, legacy_pad, size, order, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()
    res, grad, grad_estimated = gpu_g_checker.CheckSimple(op, [X, w, b], 0, [0])
    print 'Test for gpu', stride, kernel, legacy_pad, size, order, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()

Device test for 1 1 1 7 NHWC is:  True
Test for cpu 1 1 1 7 NHWC is:  True
Test for gpu 1 1 1 7 NHWC is:  True
Device test for 1 1 2 7 NHWC is:  True
Test for cpu 1 1 2 7 NHWC is:  True
Test for gpu 1 1 2 7 NHWC is:  True
Device test for 1 3 1 7 NHWC is:  True
Test for cpu 1 3 1 7 NHWC is:  True
Test for gpu 1 3 1 7 NHWC is:  True
Device test for 1 3 2 7 NHWC is:  True
Test for cpu 1 3 2 7 NHWC is:  True
Test for gpu 1 3 2 7 NHWC is:  True
Device test for 1 5 1 14 NHWC is:  True
Test for cpu 1 5 1 14 NHWC is:  True
Test for gpu 1 5 1 14 NHWC is:  True
Device test for 1 5 2 14 NHWC is:  True
Test for cpu 1 5 2 14 NHWC is:  True
Test for gpu 1 5 2 14 NHWC is:  True
Device test for 2 7 1 24 NHWC is:  True
Test for cpu 2 7 1 24 NHWC is:  True
Test for gpu 2 7 1 24 NHWC is:  True
Device test for 2 7 2 24 NHWC is:  True
Test for cpu 2 7 2 24 NHWC is:  True
Test for gpu 2 7 2 24 NHWC is:  True
Device test for 1 1 1 7 NCHW is:  True
Test for cpu 1 1 1 7 NCHW is:  True
Test for gpu 1 1 1 7 NCHW

In [3]:
# This is for max pooling layers.
test_configs = [
(2, 3, 2, 12, "NHWC"),
(2, 3, 2, 16, "NHWC"),
(1, 3, 2, 8, "NHWC"),
(1, 3, 2, 14, "NHWC"),
(2, 3, 2, 14, "NHWC"),
(1, 3, 2, 7, "NHWC"),
(2, 3, 2, 12, "NCHW"),
(2, 3, 2, 16, "NCHW"),
(1, 3, 2, 8, "NCHW"),
(1, 3, 2, 14, "NCHW"),
(2, 3, 2, 14, "NCHW"),
(1, 3, 2, 7, "NCHW"),
]

for stride, kernel, legacy_pad, size, order in test_configs:
    op = core.CreateOperator("MaxPool")(
        ["X"], ["Y", "Y_maxid"], stride=stride, kernel=kernel,
        legacy_pad=legacy_pad, order=order)
    # In order to avoid the problem of race conditions, we will do a randperm
    # so that the values will be apart at least 
    if order == "NHWC":
        X = np.random.permutation(1 * size * size * 3).reshape(1, size, size, 3).astype(np.float32) * 0.01
    else:
        X = np.random.permutation(1 * size * size * 3).reshape(1, 3, size, size).astype(np.float32) * 0.01
    res = device_checker.CheckSimple(op, [X], [0])
    print 'Device test for', stride, kernel, legacy_pad, size, order, 'is: ', res
    res, grad, grad_estimated = cpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for cpu', stride, kernel, legacy_pad, size, order, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()
    res, grad, grad_estimated = gpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for gpu', stride, kernel, legacy_pad, size, order, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()

Device test for 2 3 2 12 NHWC is:  True
Test for cpu 2 3 2 12 NHWC is:  True
Test for gpu 2 3 2 12 NHWC is:  True
Device test for 2 3 2 16 NHWC is:  True
Test for cpu 2 3 2 16 NHWC is:  True
Test for gpu 2 3 2 16 NHWC is:  True
Device test for 1 3 2 8 NHWC is:  True
Test for cpu 1 3 2 8 NHWC is:  True
Test for gpu 1 3 2 8 NHWC is:  True
Device test for 1 3 2 14 NHWC is:  True
Test for cpu 1 3 2 14 NHWC is:  True
Test for gpu 1 3 2 14 NHWC is:  True
Device test for 2 3 2 14 NHWC is:  True
Test for cpu 2 3 2 14 NHWC is:  True
Test for gpu 2 3 2 14 NHWC is:  True
Device test for 1 3 2 7 NHWC is:  True
Test for cpu 1 3 2 7 NHWC is:  True
Test for gpu 1 3 2 7 NHWC is:  True
Device test for 2 3 2 12 NCHW is:  True
Test for cpu 2 3 2 12 NCHW is:  True
Test for gpu 2 3 2 12 NCHW is:  True
Device test for 2 3 2 16 NCHW is:  True
Test for cpu 2 3 2 16 NCHW is:  True
Test for gpu 2 3 2 16 NCHW is:  True
Device test for 1 3 2 8 NCHW is:  True
Test for cpu 1 3 2 8 NCHW is:  True
Test for gpu 1 3 2 

In [4]:
# This is for average pooling layers.
checker = gradient_checker.GradientChecker(0.01, 0.05, "default")
test_configs = [
(1, 7, 1, 7),
(1, 7, 2, 7),
]

for stride, kernel, legacy_pad, size in test_configs:
    op = core.CreateOperator("AveragePool")(
        ["X"], ["Y"], stride=stride, kernel=kernel,
        legacy_pad=legacy_pad, order="NHWC")
    X = np.random.rand(2, size, size, 3).astype(np.float32)
    res = device_checker.CheckSimple(op, [X], [0])
    print 'Device test for', stride, kernel, legacy_pad, size, 'is: ', res
    res, grad, grad_estimated = cpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for cpu', stride, kernel, legacy_pad, size, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()
    res, grad, grad_estimated = gpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for gpu', stride, kernel, legacy_pad, size, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()

Device test for 1 7 1 7 is:  True
Test for cpu 1 7 1 7 is:  True
Test for gpu 1 7 1 7 is:  True
Device test for 1 7 2 7 is:  True
Test for cpu 1 7 2 7 is:  True
Test for gpu 1 7 2 7 is:  True


In [ ]:
# This is for lrn layers.
test_configs = [
(6, 10),
(3, 13),
]

for input_size, depth in test_configs:
    op = core.CreateOperator("LRN")(
        ["X"], ["Y", "Y_scale"], size=11, alpha=0.001, beta=0.5, bias=2.0, order="NHWC")
    X = np.random.rand(2, input_size, input_size, depth).astype(np.float32)
    res = device_checker.CheckSimple(op, [X], [0])
    print 'Device test for', input_size, depth, 'is: ', res
    
    res, grad, grad_estimated = cpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for cpu', input_size, depth, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()
    res, grad, grad_estimated = gpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for gpu', input_size, depth, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()

In [ ]:
# This is for depth concat layers.
test_configs = [
(3, 2, 3, 4, 5),
(4, 5, 4, 3, 2),   
]

for input_size, d1, d2, d3, d4 in test_configs:
    op = core.CreateOperator("DepthConcat")(
        ["X1", "X2", "X3", "X4"], ["Y", "Y_dims"], order="NHWC")
    Xs = [np.random.rand(2, input_size, input_size, d1).astype(np.float32),
          np.random.rand(2, input_size, input_size, d2).astype(np.float32),
          np.random.rand(2, input_size, input_size, d3).astype(np.float32),
          np.random.rand(2, input_size, input_size, d4).astype(np.float32)]
    for i in range(4):
        res = device_checker.CheckSimple(op, Xs, [0])
        print 'Device test for', input_size, d1, d2, d3, d4, 'is: ', res
        res, grad, grad_estimated = cpu_g_checker.CheckSimple(op, Xs, i, [0])
        print 'Test for cpu', input_size, d1, d2, d3, d4, 'input id', i, 'is: ', res
        if not res:
            pyplot.figure()
            pyplot.plot(grad.flat, grad_estimated.flat)
        sys.stdout.flush()
        res, grad, grad_estimated = gpu_g_checker.CheckSimple(op, Xs, i, [0])
        print 'Test for gpu', input_size, d1, d2, d3, d4, 'input id', i, 'is: ', res
        if not res:
            pyplot.figure()
            pyplot.plot(grad.flat, grad_estimated.flat)
        sys.stdout.flush()

In [ ]:
# This is for Relu layers.
checker = gradient_checker.GradientChecker(0.01, 0.05, "default")
test_configs = [
(1, 1),
(2, 1),
(1, 3, 3, 1),
(2, 3, 3, 1),
(1, 5, 5, 3),
(2, 5, 5, 3),
]

for input_size in test_configs:
    op = core.CreateOperator("Relu")(["X"], ["Y"])
    X = np.random.rand(*input_size).astype(np.float32)
    # go away from the origin point to avoid kink problems
    X += 0.01 * np.sign(X)
    X[X==0] = 0.01
    res = device_checker.CheckSimple(op, [X], [0])
    print 'Device test for', input_size, 'is: ', res
    res, grad, grad_estimated = cpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for cpu', input_size, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()
    res, grad, grad_estimated = gpu_g_checker.CheckSimple(op, [X], 0, [0])
    print 'Test for gpu', input_size, 'is: ', res
    if not res:
        pyplot.figure()
        pyplot.plot(grad.flat, grad_estimated.flat)
    sys.stdout.flush()